<a href="https://colab.research.google.com/github/yueop/AS_LAB/blob/main/main_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import sys
from google.colab import drive

# 1. 드라이브 마운트
drive.mount('/content/drive')

# 2. 경로 설정
PROJECT_PATH = '/content/drive/MyDrive/AS_LAB'
if not os.path.exists(PROJECT_PATH):
    os.makedirs(PROJECT_PATH)

# 3. 시스템 경로 추가 및 작업 디렉토리 변경
if PROJECT_PATH not in sys.path:
    sys.path.append(PROJECT_PATH)
os.chdir(PROJECT_PATH)

print(f"현재 위치: {os.getcwd()}")

Mounted at /content/drive
현재 위치: /content/drive/MyDrive/AS_LAB


In [ ]:
import shutil
import os
import sys
import yaml
from importlib import reload

# 우리가 만든 모듈들 임포트
# (파일들이 같은 폴더에 있어야 합니다)
try:
    import train
    import eval as eval_script
    import run_stats
except ImportError:
    print("오류: train.py, eval.py, run_stats.py 파일이 같은 폴더에 있는지 확인해주세요.")
    sys.exit()

def run_single_experiment(config_file):
    """
    기존처럼 한 번만 실행하는 모드
    (데이터 시각화 + 학습 + 평가 + 그래프 저장)
    """
    print(f"\n=== [단일 실험 모드] {config_file} 적용 ===")
    shutil.copy(config_file, 'config.yaml')

    # 1. 학습 실행
    print("\n>> 학습 시작 (train.py)...")
    reload(train) # 이전 설정 초기화
    train_time = train.main() # 학습 시간 반환받음 (화면 출력용은 아님)

    # 2. 평가 실행
    print("\n>> 평가 시작 (eval.py)...")
    reload(eval_script) # 이전 설정 초기화
    acc, prec, rec, f1 = eval_script.main() # 지표 반환받음

    print("\n" + "="*40)
    print(f" [단일 실행 결과 요약]")
    print(f" - 학습 시간: {train_time:.2f}초")
    print(f" - 정확도(Acc): {acc*100:.2f}%")
    print(f" - F1-score : {f1:.4f}")
    print("="*40)

def run_statistics_experiment(config_file, trials=5):
    """
    보고서용 통계 모드
    (5번 반복 + 평균/표준편차 + 시간 측정 표 출력)
    """
    print(f"\n=== [통계 실험 모드] {config_file} 적용 (반복 {trials}회) ===")
    shutil.copy(config_file, 'config.yaml')

    # run_stats.py의 함수 호출
    # (내부에서 train/eval을 반복 실행하고 표를 그려줍니다)
    reload(run_stats)
    run_stats.run_statistics('config.yaml', num_trials=trials)

# --- 메인 실행 로직 ---
if __name__ == "__main__":
    # 1. 실험할 설정 파일 선택
    # (필요한 경우 mnist나 fashionmnist로 바꿔서 실행하세요)
    target_config = 'config_fashion_improved.yaml'
    # target_config = 'config_mnist.yaml'
    # target_config = 'config_fashion.yaml'

    if not os.path.exists(target_config):
        print(f"오류: {target_config} 파일이 없습니다.")
        sys.exit()

    while True:
        print("\n" + "="*30)
        print(f" 현재 타겟 설정: {target_config}")
        print("="*30)
        print("1. 단일 실행 (기존 방식 + 시각화)")
        print("2. 통계 실행 (5회 반복 + 보고서용 표)")
        print("q. 종료")
        print("-" * 30)

        choice = input("실행할 모드를 선택하세요 (1/2/q): ").strip()

        if choice == '1':
            run_single_experiment(target_config)
        elif choice == '2':
            try:
                n = input("반복 횟수를 입력하세요 (기본 5): ").strip()
                trials = int(n) if n.isdigit() else 5
                run_statistics_experiment(target_config, trials=trials)
            except Exception as e:
                print(f"실행 중 오류 발생: {e}")
        elif choice.lower() == 'q':
            print("프로그램을 종료합니다.")
            break
        else:
            print("잘못된 입력입니다.")


 현재 타겟 설정: config_fashion_improved.yaml
1. 단일 실행 (기존 방식 + 시각화)
2. 통계 실행 (5회 반복 + 보고서용 표)
q. 종료
------------------------------
실행할 모드를 선택하세요 (1/2/q): 2
반복 횟수를 입력하세요 (기본 5): 5

=== [통계 실험 모드] config_fashion_improved.yaml 적용 (반복 5회) ===
=== [FASHION_MNIST_IMPROVED_CNN] 실험 결과 집계 시작 (5회 반복) ===

>> [Run 1/5] Seed: 42
device: cuda | model: deepcnn
[fashion_mnist] 데이터 시각화 실행
fashionmnist 샘플 시각화(클래스 당 10개)
시각화 이미지가 저장되었습니다: fashionmnist_preview.png
Scheduler Activated: StepLR (step=20, gamma=0.1)
학습을 시작합니다...
Epoch [1/60] Time: 24.47s | Train Loss: 0.5639 Train Acc: 0.7978 | Val Loss: 0.3697 Val Acc: 0.8682
Loss decreased (inf --> 0.369736). 모델을 저장합니다...
Epoch [2/60] Time: 23.07s | Train Loss: 0.3926 Train Acc: 0.8570 | Val Loss: 0.3229 Val Acc: 0.8861
Loss decreased (0.369736 --> 0.322916). 모델을 저장합니다...
Epoch [3/60] Time: 21.91s | Train Loss: 0.3553 Train Acc: 0.8718 | Val Loss: 0.2965 Val Acc: 0.8913
Loss decreased (0.322916 --> 0.296519). 모델을 저장합니다...
Epoch [4/60] Time: 22.60s | Train Loss